Ths project involves a detailed landscape character analysis of Ghana, with focus on the biophysical attributes.
This includes an analysis and assessment of precipitation-Evapotranspiration index and Land Cover datasets.
The timeframe for which data would be obtained is from (2021-01-01) to (2022- 01 - 01). Over a span of one year
Raster data for analysis and visualization would be otained from Google's Earth Engine via the python earth engine library
For visualization, geemap would be used for a visual representations of the biophysical attributes. 

BREAKDOWN OF THE TASK FOR THE LANDSCAPE CHARACTER ASSESSMENT

1. Defining the geographical extent of the region to be analyzed.
2. Acquiring relevant geospatial raster datasets for analysis
3. Preprocessing the acquired datasets for analysis
4. Performing the landscape character assessment using advanced geospatial techniques, functions and tools
5. Visualizing and communicating the findings using a suitable python visualization library 

I tried authenticating earth engine in a jupyter notebook (in this environment), but the verification box wouldn't appear to paste the code in.
Had to try the authentication in a different IDE which worked. The authentication code is in a python file name (pp.py) in this directory

In [26]:
# import necessary libraries to be used

import ee
import geemap

In [27]:
ee.Initialize(project = 'ee-reginadowlings')

In [28]:
# Defining the region of interest (Ghana)

ghana = ee.Geometry.Rectangle([-3.317, 4.730, 1.192, 11.174])


ACQUIRNG THE DATA SETS FROM EARTH ENGINE'S API. THIS DATA SETS INCLUDES ( precipitation-Evapotranspiration  index and Land Cover)

In [29]:
#Obtaining the precipitation-evapotranspiration  index of Ghana within the timeframe

# retrieving dataset from the SPEI database (Standardised Precipitation-Evapotranspiration Index database) on earth engine
pre_eva = ee.ImageCollection("CSIC/SPEI/2_9")\
.filterDate('2021-01-01' ,'2022-01-01')


# selecting data for 12-months analysis
analysis = pre_eva.select('SPEI_12_month')


#  Set the visualization ranges and color palette.
visParams = {
  'min': -2.33,
  'max':  2.33,
  'palette' : ['8b1a1a', 'de2929', 'f3641d',
    'fdc404', '9afa94', '03f2fd',
    '12adf3', '1771de', '00008b',
      
  ]

}

# selecting the region of interest against the data
gh_spei = analysis.filterBounds(ghana)


# centering the map
center_map = geemap.Map(center = [8.3463813, -1.2661157], zoom= 6 )


# lets visualize the map of Ghana
spei_layer = "SPEI_12_MONTHS"
center_map.add_layer(analysis, visParams, spei_layer)
center_map.add_layer(ghana, {'color': 'black'}, 'Ghana demarcation')

center_map

Map(center=[8.3463813, -1.2661157], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

In [30]:
# Saving the map as an html file Can be viewd by pasting link in a web browser  

out_html = "centre_map.html"

center_map.to_html(out_html)


link_to_view_map = "file:///C:/Users/acer/Documents/GIS%20Projects/landscape_character_assessment/centre_map.html"

Analysing the datasets of the precipitation-evapotranspiration index

SPEI VALUES

2.0+ = extremely wet      <>      1.5 to 1.99= very wet    <>  1.0 to 1.49 =moderately wet   <>  -.99 to .99 =near normal    <>
-1.0 to -1.49 = moderately dry    <>    -1.5 to -1.99 = severely dry     <>   -2 and less= extremely dry

In [31]:
threshold = -2

# Filter SPEI data to show only values above the threshold
spei_filtered = gh_spei.map(lambda img: img.updateMask(img.gt(threshold)))


# setting a legend 
legend_info = {
    '>2.0' : '#00008b',
    '1.5 to 1.99' : '#1771de',
    '1.0 to 1.49' : '#12adf3',
    '-.99 to .99' : '#9afa94',
    '-1.0 to -1.49': '#f3641d',
    '-1.5 to -1.99' : '#de2929',
    '< -2' : '#8b1a1a'
} 


# Display the filtered SPEI data
spei_analysis = geemap.Map(center = [8.3463813, -1.2661157], zoom= 6)

spei_analysis.addLayer(spei_filtered, visParams, 'Filtered SPEI 12 month Analysis')
spei_analysis.addLayer(ghana, {'color': 'black'}, 'ROI')
spei_analysis.add_legend(title= 'SPEI Value Legend', legend_dict= legend_info, position='topright', draggable=True, in_place =True)

spei_analysis

Map(center=[8.3463813, -1.2661157], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

INTERPRETATION

The blue color at the south-western part of Ghana indicates moderately wet as it is the forest belt of Ghana endowed with many trees and tropical rain forests. However, the orange and yellow traces at the northern part of the country known as the Savannah belt is moderately dry and relatively hot with litle to no rain, indicating a negative spei index.

In [32]:
# Saving the map as an html file. Can be viewd by pasting link in a web browser 

saving_map = "spei_analysis.html"

spei_analysis.to_html(saving_map)

# link to view map
link = "file:///C:/Users/acer/Documents/GIS%20Projects/landscape_character_assessment/spei_analysis.html"

Obtaining and analysing land cover datasets     

In [33]:
# reading the land cover data from copernicus, filtering out the roi, date frame and cloud  cover 

land_cover = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
.filterDate('2021-01-01', '2022-01-01') \
.filterBounds(ghana)



# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

land_cover = land_cover.map(apply_scale_factors)


# removing cloud contamination, defining a function to mask out cloudy pixels
def maskClouds(image):
    qa = image.select('QA_PIXEL')
    # Create a mask to identify cloudy pixels (bit 4 for clouds)
    cloud_mask = qa.bitwiseAnd(1 << 4).eq(0)
    # Apply the mask to the image
    return image.updateMask(cloud_mask)


# Applying the cloud mask function to each image in the collection
landsat_masked = land_cover.map(maskClouds)


# Take the median of the masked images to create a cloud-free composite
composite = landsat_masked.median()


# colors for map to be visualized in
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}


lc_map = geemap.Map(center = [8.3463813, -1.2661157], zoom= 6)

lc_map.add_layer(ghana, {}, 'Ghana')
lc_map.add_layer(composite, visualization, 'Landsat')

lc_map

Map(center=[8.3463813, -1.2661157], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

In [34]:
# Saving the map as an html file. Can be viewd by pasting link in a web browser 

output_html = "lc_map.html"

lc_map.to_html(output_html)

view_map = "file:///C:/Users/acer/Documents/GIS%20Projects/landscape_character_assessment/lc_map.html"

Analysing the land cover datasets.
The project expects to identify and categorize land cover types such as dense, medium, and sparse vegetation
and rocky areas. With this, the NDVI (Normalized Difference Vegetation Index) would be used to calculate the vegetation types. It is used to quantify vegetation greenness. 

Formula for calculating vegetation types using NDVI is;  NDVI = (Band 5 - Band 4) / (Band 5 + Band 4) 


NDVI = (NIR + Red) / (NIR−Red)  Where NIR (Near -Infrared Radiation) is Band 5  and Red is band 4

NB :: Very low values of NDVI (0.1 and below) correspond to barren areas of rock, sand, or snow. 
Moderate values represent shrub and grassland (0.2 to 0.3), while high values indicate temperate and tropical rainforests (0.6 to 0.8).


In [35]:
# creating a function to calculate NDVI

def ndvi(image):
    
    b5 = image.select('SR_B5')  # NIR band (band 5)
    b4 = image.select('SR_B4')  # Red band (band 4)

    # Calculate NDVI
    ndvi = b5.subtract(b4).divide(b5.add(b4)).rename('NDVI')

    return image.addBands(ndvi)


# applying the ndvi function on the land cover data
l8_ndvi = landsat_masked.map(ndvi)


# Visualizing the NDVI image
visparameters = {
    'min': -1,
    'max': 1,
    'bands': ['NDVI'],
    'palette': ['#e5f5f9', '#feb24c', '#31a354']

}


# setting a legend
legend_dict = {
    '<0.1' : '#e5f5f9',
    '>= 0.2 and <= 0.3' : '#feb24c',
    '>= 0.6 and <= 0.8' : '#31a354'

} 


# get the median of the image collection
median = l8_ndvi.median()

# plotiing the map
ndvi_map = geemap.Map(zoom= 5)

ndvi_map.add_layer(ghana, {}, 'Ghana')
ndvi_map.add_layer(median, visparameters, 'Vegetation Map')
ndvi_map.add_legend(title= 'NDVI Legend', legend_dict= legend_dict, position='bottomright', draggable=True)


ndvi_map 

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

From the map displayed above, within a span of a year, the green areas show dense vegatation or land cover. This is mostly seen at the western, southern and Eastern parts of Ghana. However, the northern part records a medium vegetation cover due to it's location, high amount of solar heat and less rain in this region. Ghana is located in the tropical forest region hence, vegetation grows all year round, which is a reason for no white color
traces in the map.

In [36]:
# Saving the map as an html file. Can be viewd by pasting link in a web browser 
output_map = "ndvi_map.html"

ndvi_map.to_html(output_map)

map_link = "file:///C:/Users/acer/Documents/GIS%20Projects/landscape_character_assessment/ndvi_map.html"

Downloading climate dataset with region of interest being GHANA. Analysis of the data would be done in QGIS  
Data would be obtained from MOD11A1.061 Terra Land Surface Temperature and Emissivity Daily Global 1km 

In [46]:
# Obtaining the surface temperature of Ghana from September 2023 - December 2023 and January 2024 - April 2024
# Calculating the mean temperatures and clippping the region of interest on the image collection 

# temperature for 2023
temperature_dataset23 = ee.ImageCollection("MODIS/061/MOD11A1")\
.filterDate('2023-09-01', '2023-12-31') \
.filterBounds(ghana)\
.map(lambda image: image.clip(ghana))

# selecting bands and the mean temperature of the time period
surface_temperature = temperature_dataset23.select('LST_Day_1km')
mean_temp23 = surface_temperature.mean()


# visualizing the data as a map
temperature_visual = {
    'min': 13000.0,
    'max': 16500.0,
    'band': mean_temp23,
    'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]
    
}
 
# setting map parameters

temp_map = geemap.Map(center = [8.3463813, -1.2661157], zoom= 6)

temp_map.add_layer(mean_temp23, temperature_visual, 'Land Surface Temperature')


temp_map

Map(center=[8.3463813, -1.2661157], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

In [47]:
# temperature for 2024

temperature_dataset24 = ee.ImageCollection("MODIS/061/MOD11A1")\
.filterDate('2024-01-01', '2024-04-30') \
.filterBounds(ghana)\
.map(lambda image: image.clip(ghana))

# selecting bands and getting the mean temperature of the time period
surface_temperature24 = temperature_dataset24.select('LST_Day_1km')
mean_temp24 = surface_temperature24.mean()

temp_map24 = geemap.Map(center = [8.3463813, -1.2661157], zoom= 6)

temp_map24.add_layer(surface_temperature, temperature_visual, 'Land Surface Temperature')

temp_map24

Map(center=[8.3463813, -1.2661157], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

Exporting mean temperatures for 2023

In [50]:
export_task = ee.batch.Export.image.toDrive(**{
    'image': mean_temp23,
    'description': 'Mean Temperature of 2023',
    'folder': 'Project',
    'scale': 1000,
    'region': ghana,
    'fileFormat': 'GeoTIFF',
    'crs': 'EPSG:4326'
})
export_task.start()


Exporting the mean temperature for 2024

In [51]:

export_task = ee.batch.Export.image.toDrive(**{
        'image': mean_temp24,
        'description': 'Mean Temperature of 2024',
        'folder': 'Project',
        'scale': 1000,
        'region': ghana,
        'fileFormat': 'GeoTIFF',
        'crs': 'EPSG:4326'
})
export_task.start()

In [ ]:
# Exporting image collection 



# def export_image(image, index):
#     export_task = ee.batch.Export.image.toDrive(**{
#         'image': image,
#         'description': f'LST_Day_1km_Clipped_{index}',
#         'folder': 'Proect',
#         'fileNamePrefix': f'LST_Day_1km_Clipped_{index}',
#         'scale': 1000,
#         'region': ghana,
#         'fileFormat': 'GeoTIFF',
#         'crs': 'EPSG:4326'
#     })
#     export_task.start()
#     print(f"Export task started for image {index}")

# # Export each image in the collection
# image_list =mean_temp24.toList(mean_temp24.size())
# num_images = image_list.size().getInfo()
# print(f"Number of images to export: {num_images}")

# for i in range(num_images):
#     image = ee.Image(image_list.get(i))
#     export_image(image, i)

# print("All export tasks started. Check your Google Drive for the exported images.")